In [ ]:

#--------------------------------------------------------------------
# Exercises 4.

# Exercise 4.1.

# Titanic passengers data – 1310 observations and 15 variables:
# passenger_id – Unique passenger id
# pclass – Ticket class (1 = 1st, 2 = 2nd, 3 = 3rd)
# survived – Survival (0 = No, 1 = Yes)
# name – Name and SUrname
# sex – Sex (0 = Male, 1 = Female)
# age – Age in years
# sibsp – # of siblings / spouses aboard the Titanic
# parch – # of parents / children aboard the Titanic
# ticket – Ticket number
# fare – Passenger fare
# cabin – Cabin number
# embarked – Port of Embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)
# boat – Lifeboat (if survived)
# body – Body number (if did not survive and body was recovered)
# home.dest – Home/Destination
# 
# Re-run your best models for all algorithms for 5-fold CV. 
# Check the stability of results for repeated K-fold
# Check in repeated k-fold CV if adding stratification changes your results (stability)
# Check if you didnt overfit in your models. Check if you can imrpove you validation score.

In [11]:
import pandas as pd
from sklearn.model_selection import KFold
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, RepeatedKFold, StratifiedKFold

đọc file titanic.csv + preprocessing data bằng cách fillna 1 vài cột dữliệu số bằng median, còn dữ liệu chữ thì chuyển sang dạng category để phù hợp với model regression

In [ ]:

a = pd.read_csv("titanic.csv")
a['fare'] = a['fare'].fillna(a['fare'].median())
a['embarked'] = a['embarked'].fillna(a['embarked'].mode()[0])
a['age'] = a['age'].fillna(a['age'].median())
a['embarked'] = a['embarked'].map({'C': 'Cherbourg', 'Q': 'Queenstown', 'S': 'Southampton'}).astype('category')
a['sex'] = a['sex'].map({0: 'Male', 1: 'Female'}).astype('category')
a['pclass'] = a['pclass'].map({1: '1st', 2: '2nd', 3: '3rd'}).astype('category')
pd.set_option("display.max_columns",50)
a.head()




,passenger_id,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1st,1,"Allen, Miss. Elisabeth Walton",Female,29.0000,0,0,24160,211.3375,B5,Southampton,2,NaN,"St Louis, MO"
1,2,1st,1,"Allison, Master. Hudson Trevor",Male,0.9167,1,2,113781,151.5500,C22 C26,Southampton,11,NaN,"Montreal, PQ / Chesterville, ON"
2,3,1st,0,"Allison, Miss. Helen Loraine",Female,2.0000,1,2,113781,151.5500,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,4,1st,0,"Allison, Mr. Hudson Joshua Creighton",Male,30.0000,1,2,113781,151.5500,C22 C26,Southampton,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,5,1st,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",Female,25.0000,1,2,113781,151.5500,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON"


các chỉ số R-squared rất thấp (đều < 0.11) và RMSE dao động quanh mức 0.47 cho thấy mô hình hồi quy tuyến tính không hiệu quả và gần như không giải thích được sự biến thiên của biến mục tiêu survived

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
a['age'] = a['age'].fillna(a['age'].median())
a['fare'] = a['fare'].fillna(a['fare'].median())

if a['survived'].dtype == 'object' or a['survived'].dtype.name == 'category':
    a['survived'] = a['survived'].map({'Yes': 1, 'No': 0, 1: 1, 0: 0})

X = a.select_dtypes(include=[np.number]).drop(columns=['survived', 'passenger_id', 'body'], errors='ignore')
y = a['survived']

a_clean = pd.concat([X, y], axis=1).sample(frac=1, random_state=42).reset_index(drop=True)
X = a_clean.drop(columns=['survived'])
y = a_clean['survived']

k_fold = 10
fold_size = len(a_clean) // k_fold

for k in range(k_fold):
    start = k * fold_size
    end = (k + 1) * fold_size
    
    X_test = X.iloc[start:end]
    y_test = y.iloc[start:end]
    X_train = pd.concat([X.iloc[:start], X.iloc[end:]])
    y_train = pd.concat([y.iloc[:start], y.iloc[end:]])
    
    if X_train.isnull().values.any():
        continue

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"Fold {k + 1}: R-squared = {r2:.4f}, RMSE = {rmse:.4f}")

Fold 1: R-squared = 0.0697, RMSE = 0.4710
Fold 2: R-squared = 0.0431, RMSE = 0.4807
Fold 3: R-squared = 0.0635, RMSE = 0.4690
Fold 4: R-squared = 0.0766, RMSE = 0.4572
Fold 5: R-squared = 0.0418, RMSE = 0.4417
Fold 6: R-squared = 0.0695, RMSE = 0.4753
Fold 7: R-squared = 0.0519, RMSE = 0.4785
Fold 8: R-squared = 0.0481, RMSE = 0.4665
Fold 9: R-squared = 0.1097, RMSE = 0.4650
Fold 10: R-squared = 0.0552, RMSE = 0.4802


Kết quả cho thấy model đạt độ chính xác khá cao và ổn định với chỉ số AUC trung bình khoảng 0.84, trong đó các yếu tố như giới tính, hạng vé và độ tuổi có ảnh hưởng rõ rệt nhất đến biến survived


In [ ]:
# Re-run your best models for all algorithms for 5-fold CV. 
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = []
for x, y in kf.split(a):
    formula = 'survived ~ pclass + sex + age + sibsp + parch + fare + embarked'
    model = sm.GLM.from_formula(formula = formula, data = a.iloc[x], family = sm.families.Binomial()).fit()
    preds = model.predict(a.iloc[y])
    auc = roc_auc_score(a.iloc[y]['survived'], preds)
    scores.append(auc)
print(scores)
model.summary()
    

[0.8574329096045199, 0.8435488909715714, 0.8406820640071261, 0.8580259222333001, 0.8107173200631144]


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               survived   No. Observations:                 1048
Model:                            GLM   Df Residuals:                     1038
Model Family:                Binomial   Df Model:                            9
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -464.48
Date:                Mon, 12 Jan 2026   Deviance:                       928.96
Time:                        08:26:51   Pearson chi2:                 1.05e+03
No. Iterations:                     5   Pseudo R-squ. (CS):             0.3538
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   3.8353      0.436      8.788      0.000       2.980       4.691
pclass[T.2nd]              -1.0030      0.278     -3.604      0.000      -1.549      -0.458
pclass[T.3rd]              -1.8789      0.276     -6.796      0.000      -2.421      -1.337
sex[T.Male]                -2.7385      0.183    -14.998      0.000      -3.096      -2.381
embarked[T.Queenstown]     -0.5036      0.347     -1.452      0.146      -1.183       0.176
embarked[T.Southampton]    -0.6286      0.215     -2.922      0.003      -1.050      -0.207
age                        -0.0323      0.007     -4.444      0.000      -0.046      -0.018
sibsp                      -0.2744      0.099     -2.783      0.005      -0.468      -0.081
parch                       0.0038      0.115      0.033      0.974      -0.222       0.230
fare                        0.0015      0.002      0.708      0.479      -0.003       0.005
===========================================================================================
"""

Kết quả AUC đạt mức trung bình tốt và ổn định (dao động từ 0.76 đến 0.92), cho thấy model có khả năng tổng quát hóa tốt và không bị ảnh hưởng quá lớn bởi cách chia dữ liệu



In [ ]:
# Check the stability of results for repeated K-fold
bold = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = []
for x, y in bold.split(a):
    model = sm.GLM.from_formula(formula, data=a.iloc[x], family=sm.families.Binomial()).fit()
    preds = model.predict(a.iloc[y])
    scores.append(roc_auc_score(a.iloc[y]['survived'], preds))
print(scores)
model.summary()

[0.8533088235294117, 0.8760765550239235, 0.7757352941176471, 0.8295061728395061, 0.8909090909090909, 0.8443963443963444, 0.9213554987212276, 0.8266817269076304, 0.8215762273901809, 0.7735947712418302, 0.7761044176706827, 0.8641946150212565, 0.8183357522980164, 0.8987745098039216, 0.7594497607655502, 0.9002026342451874, 0.829923273657289, 0.8528395061728394, 0.8869565217391306, 0.8337215320910973, 0.8490374873353597, 0.8377302140368342, 0.8278554778554779, 0.8153846153846155, 0.7942152678994784, 0.8692269076305221, 0.8480818414322251, 0.8134498480243161, 0.8845679012345679, 0.8445023579051874]


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               survived   No. Observations:                 1179
Model:                            GLM   Df Residuals:                     1169
Model Family:                Binomial   Df Model:                            9
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -541.51
Date:                Mon, 12 Jan 2026   Deviance:                       1083.0
Time:                        08:26:55   Pearson chi2:                 1.21e+03
No. Iterations:                     5   Pseudo R-squ. (CS):             0.3366
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   3.8630      0.396      9.761      0.000       3.087       4.639
pclass[T.2nd]              -0.9589      0.254     -3.768      0.000      -1.458      -0.460
pclass[T.3rd]              -1.9987      0.253     -7.903      0.000      -2.494      -1.503
sex[T.Male]                -2.5504      0.168    -15.217      0.000      -2.879      -2.222
embarked[T.Queenstown]     -0.3958      0.313     -1.263      0.207      -1.010       0.219
embarked[T.Southampton]    -0.6662      0.197     -3.381      0.001      -1.052      -0.280
age                        -0.0343      0.007     -5.092      0.000      -0.047      -0.021
sibsp                      -0.2823      0.097     -2.904      0.004      -0.473      -0.092
parch                       0.0249      0.096      0.261      0.794      -0.162       0.212
fare                        0.0009      0.002      0.475      0.635      -0.003       0.005
===========================================================================================
"""

 Kết quả AUC đạt mức trung bình ~0.84 và khá đồng nhất giữa các fold, cho thấy model hoạt động tin cậy và có khả năng tổng quát hóa tốt trên các tập dữ liệu con khác nhau.

In [ ]:
# Check in repeated k-fold CV if adding stratification changes your results (stability)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []
for x, y in skf.split(a, a['survived']):
    model = sm.GLM.from_formula(formula, data=a.iloc[x], family=sm.families.Binomial()).fit()
    preds = model.predict(a.iloc[y])
    scores.append(roc_auc_score(a.iloc[y]['survived'], preds))
print(scores)
model.summary()

[0.8245061728395061, 0.8725925925925926, 0.7973148148148147, 0.8525617283950617, 0.8531366459627329]


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               survived   No. Observations:                 1048
Model:                            GLM   Df Residuals:                     1038
Model Family:                Binomial   Df Model:                            9
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -485.06
Date:                Mon, 12 Jan 2026   Deviance:                       970.11
Time:                        08:31:51   Pearson chi2:                 1.06e+03
No. Iterations:                     5   Pseudo R-squ. (CS):             0.3324
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   3.8177      0.430      8.876      0.000       2.975       4.661
pclass[T.2nd]              -0.8389      0.272     -3.082      0.002      -1.372      -0.305
pclass[T.3rd]              -1.9002      0.274     -6.945      0.000      -2.437      -1.364
sex[T.Male]                -2.5418      0.177    -14.348      0.000      -2.889      -2.195
embarked[T.Queenstown]     -0.5483      0.335     -1.635      0.102      -1.206       0.109
embarked[T.Southampton]    -0.6017      0.208     -2.898      0.004      -1.009      -0.195
age                        -0.0352      0.007     -5.018      0.000      -0.049      -0.021
sibsp                      -0.2618      0.099     -2.653      0.008      -0.455      -0.068
parch                      -0.0315      0.097     -0.326      0.744      -0.221       0.158
fare                        0.0011      0.002      0.479      0.632      -0.003       0.005
===========================================================================================
"""

Với khoảng cách sai lệch cực nhỏ (chỉ 0.0037), kết quả khẳng định model không bị overfitting và có khả năng tổng quát hóa rất tốt trên dữ liệu thực tế.

In [ ]:
# Check if you didnt overfit in your models. Check if you can imrpove you validation score.
train, test = [], []
for x, y in bold.split(a):
    df_train = a.iloc[x]
    df_test = a.iloc[y]
    model = sm.GLM.from_formula(formula, data=df_train, family=sm.families.Binomial()).fit()
    preds = model.predict(df_test)
    test.append(roc_auc_score(df_test['survived'], preds))
    train.append(roc_auc_score(df_train['survived'], model.predict(df_train)))
print(f"mean train: {np.mean(train):.4f}")
print(f"mean test": {np.mean(test):.4f}")
print(f"generalization gap: {np.mean(train) - np.mean(test):.4f}")

Mean Train AUC: 0.8443
Mean Valid AUC: 0.8406
Gap: 0.0037


In [ ]:

# Exercise 4.2.
# a Quality Data Set: "data/as.csv"
# source: https://archive.ics.uci.edu/ml/datasets/a+quality
# The file contains data on samples of white and red Portuguese a 
# Vinho Verde. 
# Various physico-chemical characteristics of individual samples
# are available as well as a quality scores on a point scale (0-10) 
# made by specialists.

# Re-run your best models for all algorithms for 5-fold CV. 
# Check the stability of results for repeated K-fold
# Check in repeated k-fold CV if adding stratification changes your results (stability).
# Compare the effect of stratification with titanic problem.
# Check if you didnt overfit in your models. Check if you can imrpove you validation score.




In [10]:
import pandas as pd
from sklearn.model_selection import KFold
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, RepeatedKFold, StratifiedKFold
from sklearn.metrics import mean_squared_error

In [ ]:
a = pd.read_csv("as.csv")
pd.set_option("display.max_columns",50)
a.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,type,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,red,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,red,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,red,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,red,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,red,5


Kết quả chỉ số RMSE rất ổn định quanh mức 0.73 - 0.74, cho thấy model có độ tin cậy cao và khả năng dự báo đồng nhất trên toàn bộ tập dữ liệu

In [ ]:
# Re-run your best models for all algorithms for 5-fold CV. 
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = []
for x, y in kf.split(a):
    formula = 'quality ~ fixed_acidity + volatile_acidity + citric_acid + residual_sugar + chlorides + free_sulfur_dioxide + total_sulfur_dioxide + density + pH + sulphates + alcohol'
    model = sm.GLM.from_formula(formula = formula, data = a.iloc[x], family = sm.families.Gaussian()).fit()    
    preds = model.predict(a.iloc[y])
    actuals = a.iloc[y]['quality']
    rmse = np.sqrt(mean_squared_error(actuals, preds))
    scores.append(rmse)
model.summary()
print(score)
    

[np.float64(0.7393892357611483), np.float64(0.7351170770242444), np.float64(0.7308378130356215), np.float64(0.7390379535487489), np.float64(0.7376454698947655)]


Kết quả RMSE trung bình đạt 0.7364 với độ lệch chuẩn cực thấp (0.0085), khẳng định model hoạt động rất ổn định và có sai số dự báo đồng nhất trên các tập dữ liệu khác nhau

In [ ]:
# Check the stability of results for repeated K-fold
b = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
scores = []
formula = 'quality ~ fixed_acidity + volatile_acidity + citric_acid + residual_sugar + chlorides + free_sulfur_dioxide + total_sulfur_dioxide + density + pH + sulphates + alcohol'
for x, y in b.split(a):
    model = sm.GLM.from_formula(formula=formula, data=a.iloc[x], family=sm.families.Gaussian()).fit()
    preds = model.predict(a.iloc[y])
    rmse = np.sqrt(mean_squared_error(a.iloc[y]['quality'], preds))
    scores.append(rmse)
print(f"rmse: {scores}")
print(f"mean rmse: {np.mean(scores):.4f}, std rmse: {np.std(scores):.4f}")

rmse: [np.float64(0.7393892357611483), np.float64(0.7351170770242444), np.float64(0.7308378130356215), np.float64(0.7390379535487489), np.float64(0.7376454698947655), np.float64(0.7175594553272401), np.float64(0.7247243485403622), np.float64(0.7418459452541957), np.float64(0.7515540922800457), np.float64(0.7454659044618098), np.float64(0.7345432223816662), np.float64(0.746529298997779), np.float64(0.7399507884772227), np.float64(0.7268376171899051), np.float64(0.7354256865338961)]
mean rmse: 0.7364, std rmse: 0.0085


Kết quả cho thấy model không bị overfitting và việc thêm các biến phức tạp đã giúp giảm sai số RMSE từ 0.7364 xuống 0.7353, cho thấy hiệu quả dự báo được cải thiện nhẹ

In [30]:
# Check if you didnt overfit in your models. Check if you can imrpove you validation score.
formula2 = 'quality ~ fixed_acidity + volatile_acidity + citric_acid + residual_sugar + chlorides + free_sulfur_dioxide + total_sulfur_dioxide + density + pH + sulphates + alcohol'
B = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
train, val = [], []

for x, y in B.split(a):
    m = sm.GLM.from_formula(formula2, data=a.iloc[x], family=sm.families.Gaussian()).fit()
    train.append(np.sqrt(mean_squared_error(a.iloc[x]['quality'], m.predict(a.iloc[x]))))
    val.append(np.sqrt(mean_squared_error(a.iloc[y]['quality'], m.predict(a.iloc[y]))))
print(f"a - train rmse: {np.mean(train):.4f}, Val RMSE: {np.mean(val):.4f}")
formula3 = formula2 + ' + alcohol:volatile_acidity + np.power(alcohol, 2)'
val_imp = []
for x, y in B.split(a):
    m_imp = sm.GLM.from_formula(formula3, data=a.iloc[x], family=sm.families.Gaussian()).fit()
    val_imp.append(np.sqrt(mean_squared_error(a.iloc[y]['quality'], m_imp.predict(a.iloc[y]))))

print(f"a new val rmse: {np.mean(val_imp):.4f}")

a - train rmse: 0.7344, Val RMSE: 0.7364
a new val rmse: 0.7353
